In [87]:
import pandas as pd
import numpy as np
import ast
import xlsxwriter

In [22]:
# Codon utilities
codon_list = [None] * 61
codon_list[0] = "AAA"
codon_list[1] = "AAC"
codon_list[2] = "AAG"
codon_list[3] = "AAU"
codon_list[4] = "ACA"
codon_list[5] = "ACC"
codon_list[6] = "ACG"
codon_list[7] = "ACU"
codon_list[8] = "AGA"
codon_list[9] = "AGC"
codon_list[10] = "AGG"
codon_list[11] = "AGU"
codon_list[12] = "AUA"
codon_list[13] = "AUC"
codon_list[14] = "AUG"
codon_list[15] = "AUU"
codon_list[16] = "CAA"
codon_list[17] = "CAC"
codon_list[18] = "CAG"
codon_list[19] = "CAU"
codon_list[20] = "CCA"
codon_list[21] = "CCC"
codon_list[22] = "CCG"
codon_list[23] = "CCU"
codon_list[24] = "CGA"
codon_list[25] = "CGC"
codon_list[26] = "CGG"
codon_list[27] = "CGU"
codon_list[28] = "CUA"
codon_list[29] = "CUC"
codon_list[30] = "CUG"
codon_list[31] = "CUU"
codon_list[32] = "GAA"
codon_list[33] = "GAC"
codon_list[34] = "GAG"
codon_list[35] = "GAU"
codon_list[36] = "GCA"
codon_list[37] = "GCC"
codon_list[38] = "GCG"
codon_list[39] = "GCU"
codon_list[40] = "GGA"
codon_list[41] = "GGC"
codon_list[42] = "GGG"
codon_list[43] = "GGU"
codon_list[44] = "GUA"
codon_list[45] = "GUC"
codon_list[46] = "GUG"
codon_list[47] = "GUU"
# UAA is a stop codon
codon_list[48] = "UAC"
# UAG is a stop codon
codon_list[49] = "UAU"
codon_list[50] = "UCA"
codon_list[51] = "UCC"
codon_list[52] = "UCG"
codon_list[53] = "UCU"
# UGA is a stop codon
codon_list[54] = "UGC"
codon_list[55] = "UGG"
codon_list[56] = "UGU"
codon_list[57] = "UUA"
codon_list[58] = "UUC"
codon_list[59] = "UUG"
codon_list[60] = "UUU"

def get_codon_num(codon_string):
    try:
        return codon_list.index(codon_string)
    except ValueError:
        return -1   # stop codon

def get_codon_string(codon_num):
    return codon_list[codon_num]

In [64]:
def get_riboseq_val(chromosome_df, coord):
    """Given the chromosome data frame and the integer coordinate, return the Riboseq
    value associated with that coordinate value, or 0 if it doesn't exist inside the dataframe."""
    rval = chromosome_df[(chromosome_df['Start'] <= coord) & (chromosome_df['End'] > coord)]
    if rval.empty:
        return 0
    return rval['Riboseq'].values[0]

def nucleotide_seq_to_codon_nums(nucleotide_seq):
    """Takes in a string consisting of the nucleotide sequence, and returns np array of codon numbers."""
    codon_seq_length = len(nucleotide_seq) // 3
    codon_num_list = np.zeros(codon_seq_length, dtype = int)
    for i in range(codon_seq_length):
        codon_num_list[i] = get_codon_num(nucleotide_seq[(3*i) : (3*i+3)])
    return codon_num_list

In [88]:
def harvest_rho(bedgraph_file_path, tsv_file_path):
    """Given the file paths to the bedGraph and TSV files, for every gene in the human genome write into an
    Excel file the codon sequence for that gene, as well as the raw RefSeq values."""
    bedgraph = pd.read_csv(bedgraph_file_path, sep='\t', header = None, names = ['Chromosome', 'Start', 'End', 'Riboseq'])
    tsv = pd.read_csv(tsv_file_path, sep='\t', header = None, \
        names = ['Gene Name','Transcript Name','Chromosome','+/-',"5'-UTR","Exon Seq","3'-UTR","5'-UTR Coords","Coding Coords","3'-UTR Coords"])

    # Below is a dictionary of chromosome data frames.
    chromosome_dfs = dict()
    for i in range(1,23):
        chr_string = "chr" + str(i)
        chromosome_dfs[chr_string] = bedgraph.loc[bedgraph['Chromosome'] == chr_string]
    chromosome_dfs['chrX'] = bedgraph.loc[bedgraph['Chromosome'] == 'chrX']
    chromosome_dfs['chrY'] = bedgraph.loc[bedgraph['Chromosome'] == 'chrY']
    
    num_genes = len(tsv.index)
    
    codon_str_sequences = np.asarray(tsv['Exon Seq'].values)   # np array of all the codon sequences, each as a string
    codon_num_sequences = [None] * num_genes   # list of np arrays, where each array is numeric codon sequence
    gene_names = np.asarray(tsv['Gene Name'].values)   # np array of all the gene names, each as strings
    mapped_coordinates = [ast.literal_eval(s) for s in tsv['Coding Coords'].values]   # list of lists, each holding coding coordinates
    senses = np.asarray(tsv['+/-'].values)   # np array of +/- values, depending on which strand of DNA gene is on
    chromosomes = np.asarray(tsv['Chromosome'].values)   # np array of chromosome locations of the genes
    raw_rho_vectors = [None] * num_genes
    
    # First, get all the codon sequences
    for g in range(num_genes):
        codon_num_sequences[g] = nucleotide_seq_to_codon_nums(codon_str_sequences[g])
    
    # Next, assemble together all of the raw rho vectors
    for g in range(num_genes):
        if senses[g] == '-':
            continue   # skip those genes for now
        seq_length = len(codon_str_sequences[g])   # number of RNA nucleotides inside coding transcript
        raw_rho_vector = np.zeros(seq_length // 3)   # want integer number of indices
        contiguous_coordinates = []   # will hold a list, in order, of all the coordinates for the coding sequence
        coordinate_list = mapped_coordinates[g]
        for coordinate_str in coordinate_list:
            elems = coordinate_str.split(" ")
            begin, end = int(elems[2]), int(elems[3])
            contiguous_coordinates.extend(list(range(begin, end)))
        for i in range(seq_length // 3):
            raw_rho_vector[i] = np.mean([get_riboseq_val(chromosome_dfs[chromosomes[g]], contiguous_coordinates[3*i+c]) for c in range(0,3)])
        raw_rho_vectors[g] = raw_rho_vector
        
    # Write outputs to Excel file
    workbook = xlsxwriter.Workbook('Human Genome Input.xlsx')
    sheet = workbook.add_worksheet()
    for g in range(num_genes):
        row_index = 3 * g
        sheet.write(row_index, 0, gene_names[g])
        for column, value in enumerate(codon_num_sequences[g].tolist()):
            sheet.write(row_index+1, column, value)
        for column, value in enumerate(raw_rho_vectors[g].tolist()):
            sheet.write(row_index+2, column, value)
    workbook.close()
            
    # return codon_num_sequences, raw_rho_vectors

In [90]:
# codon_num_sequences, raw_rho_vectors = harvest_rho("GM18502.bedGraph", "GENCODE.v34.transcript_sequences.sanitized")
codon_num_sequences, raw_rho_vectors = harvest_rho("GM18502.bedGraph", "test.tsv")

0 14
1 51
2 2
3 42
4 13
5 30
6 18
7 46
8 19
9 23
10 22
11 13
12 54
13 33
14 54
15 22
16 41
17 54
18 24
19 12
20 51
21 51
22 22
23 46
24 1
25 26
26 42
27 26
28 30
29 36
30 33
31 2
32 10
33 4
34 45
35 37
36 30
37 23
38 37
39 37
40 26
41 1
42 30
43 2
44 2
45 34
46 24
47 7
48 21
49 9
50 58
51 53
52 37
53 9
54 35
55 43
56 33
57 9
58 33
59 42
60 11
61 41
62 21
63 5
64 56
65 42
66 26
67 26
68 20
69 41
70 59
71 2
72 18
73 34
74 35
75 43
76 22
77 17
78 13
79 27
80 13
81 14
82 2
83 8
84 8
85 45
86 17
87 5
88 17
89 55
90 33
91 46
92 1
93 13
94 53
95 58
96 24
97 34
98 38
99 51
100 54
101 9
102 18
103 33
104 41
105 1
106 31
107 21
108 5
109 29
110 12
111 51
112 9
113 45
114 17
115 25
116 9
117 25
118 17
119 29
120 47
121 14
122 21
123 34
124 19
125 18
126 9
127 25
128 56
129 32
130 58
131 18
132 8
133 41
134 9
135 30
136 34
137 15
138 41
139 30
140 24
141 21
142 37
143 43
144 33
145 30
146 59
147 41
148 2
149 10
150 30
151 41
152 25
153 51
154 21
155 27
156 13
157 9
158 9
159 33
160 54
161 60
162 5

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-90-a3647c25eb12>", line 2, in <module>
    codon_num_sequences, raw_rho_vectors = harvest_rho("GM18502.bedGraph", "test.tsv")
TypeError: cannot unpack non-iterable NoneType object

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2039, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Anaconda3\

TypeError: cannot unpack non-iterable NoneType object

In [75]:
codon_num_sequences

[array([14, 51,  2, 42, 13, 30, 18, 46, 19, 23, 22, 13, 54, 33, 54, 22, 41,
        54, 24, 12, 51, 51, 22, 46,  1, 26, 42, 26, 30, 36, 33,  2, 10,  4,
        45, 37, 30, 23, 37, 37, 26,  1, 30,  2,  2, 34, 24,  7, 21,  9, 58,
        53, 37,  9, 35, 43, 33,  9, 33, 42, 11, 41, 21,  5, 56, 42, 26, 26,
        20, 41, 59,  2, 18, 34, 35, 43, 22, 17, 13, 27, 13, 14,  2,  8,  8,
        45, 17,  5, 17, 55, 33, 46,  1, 13, 53, 58, 24, 34, 38, 51, 54,  9,
        18, 33, 41,  1, 31, 21,  5, 29, 12, 51,  9, 45, 17, 25,  9, 25, 17,
        29, 47, 14, 21, 34, 19, 18,  9, 25, 56, 32, 58, 18,  8, 41,  9, 30,
        34, 15, 41, 30, 24, 21, 37, 43, 33, 30, 59, 41,  2, 10, 30, 41, 25,
        51, 21, 27, 13,  9,  9, 33, 54, 60, 50, 34,  2, 10, 36, 24,  9, 32,
        52, 23, 16, 34, 38, 30, 30, 30, 22, 26, 34, 30, 42, 21,  9, 14, 37,
        22, 34, 33, 19, 48, 25, 26, 31, 46, 50, 36, 30,  9, 34, 37,  9,  5,
        60, 34, 33, 23, 18, 25, 29, 48, 17, 30, 41, 29, 21,  9, 17, 43, 34,
        33, 

In [76]:
raw_rho_vectors

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

In [66]:
['chr1 + 925941 926013', 'chr1 + 930154 930336', 'chr1 + 931038 931089', 'chr1 + 935771 935896', 'chr1 + 939039 939129', 'chr1 + 939274 939460', 'chr1 + 941143 941306', 'chr1 + 942135 942251', 'chr1 + 942409 942488', 'chr1 + 942558 943058', 'chr1 + 943252 943377', 'chr1 + 943697 943808', 'chr1 + 943907 944153']

['chr1 + 925941 926013',
 'chr1 + 930154 930336',
 'chr1 + 931038 931089',
 'chr1 + 935771 935896',
 'chr1 + 939039 939129',
 'chr1 + 939274 939460',
 'chr1 + 941143 941306',
 'chr1 + 942135 942251',
 'chr1 + 942409 942488',
 'chr1 + 942558 943058',
 'chr1 + 943252 943377',
 'chr1 + 943697 943808',
 'chr1 + 943907 944153']

0


In [ ]:
def get_riboseq_value(chromosome_df, coord):
    """Given the chromosome data frame and the integer coordinate, return the Riboseq
    value associated with that coordinate value, or 0 if it doesn't exist inside the dataframe."""
    rval = chromosome_df[(chromosome_df['Start'] <= coord) & (chromosome_df['End'] > coord)]
    if rval.empty:
        return 0
    return rval['Riboseq'].values[0]